In [1]:
import pandas as pd
import numpy as np

In [2]:
courier_actions = pd.read_csv("courier_actions.csv")

In [3]:
courier_actions.head()

,courier_id,order_id,action,time
0,12,1,accept_order,24/08/22 01:52
1,12,1,deliver_order,24/08/22 02:15
2,156,2,accept_order,24/08/22 06:37
3,156,2,deliver_order,24/08/22 06:56
4,49,3,accept_order,24/08/22 07:35


In [4]:
courier_actions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116211 entries, 0 to 116210
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   courier_id  116211 non-null  int64 
 1   order_id    116211 non-null  int64 
 2   action      116211 non-null  object
 3   time        116211 non-null  object
dtypes: int64(2), object(2)
memory usage: 3.5+ MB


In [5]:
# Alteração na tabela para combinar os tempos de aceitar e entregar para cada pedido
courier_actions = courier_actions.pivot(index=['courier_id', 'order_id'], columns='action', values='time').reset_index()

# Renomear as colunas
courier_actions.columns = ['courier_id', 'order_id', 'accept_time', 'deliver_time']

# Ordernar pelo id do pedido
courier_actions = courier_actions.sort_values(by='order_id').reset_index()

# Remover a antiga coluna de index
del courier_actions['index']

# Remover valores NaN e infinitos (inf)
courier_actions = courier_actions.replace([np.inf, -np.inf], np.nan).dropna()

# Converter as colunas em objetos de data e tempo
courier_actions['accept_time'] = pd.to_datetime(courier_actions['accept_time'], format='%d/%m/%y %H:%M')
courier_actions['deliver_time'] = pd.to_datetime(courier_actions['deliver_time'], format='%d/%m/%y %H:%M')

# Calcular a diferença de tempo entre accept_time e deliver_time
courier_actions['waiting_time'] = courier_actions['deliver_time'] - courier_actions['accept_time']

# Converter a diferença de tempo em minutos (número inteiro)
courier_actions['waiting_time_minutes'] = (courier_actions['waiting_time'].dt.total_seconds() / 60).astype(int)

# Exibir o novo DataFrame com as alterações
courier_actions.head()

,courier_id,order_id,accept_time,deliver_time,waiting_time,waiting_time_minutes
0,12,1,2022-08-24 01:52:00,2022-08-24 02:15:00,0 days 00:23:00,23
1,156,2,2022-08-24 06:37:00,2022-08-24 06:56:00,0 days 00:19:00,19
2,49,3,2022-08-24 07:35:00,2022-08-24 07:54:00,0 days 00:19:00,19
3,134,4,2022-08-24 10:39:00,2022-08-24 10:58:00,0 days 00:19:00,19
4,106,5,2022-08-24 12:34:00,2022-08-24 12:59:00,0 days 00:25:00,25


In [6]:
courier_actions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56616 entries, 0 to 59594
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   courier_id            56616 non-null  int64          
 1   order_id              56616 non-null  int64          
 2   accept_time           56616 non-null  datetime64[ns] 
 3   deliver_time          56616 non-null  datetime64[ns] 
 4   waiting_time          56616 non-null  timedelta64[ns]
 5   waiting_time_minutes  56616 non-null  int32          
dtypes: datetime64[ns](2), int32(1), int64(2), timedelta64[ns](1)
memory usage: 2.8 MB


In [7]:
courier_actions.describe()

,courier_id,order_id,waiting_time,waiting_time_minutes
count,56616.000000,56616.000000,56616,56616.000000
mean,989.086972,29822.735199,0 days 00:19:57.334675710,19.955578
std,726.935825,17213.717035,0 days 00:03:02.864825310,3.047747
min,1.000000,1.000000,0 days 00:00:00,0.000000
25%,388.000000,14913.750000,0 days 00:18:00,18.000000
50%,819.000000,29849.500000,0 days 00:20:00,20.000000
75%,1527.000000,44722.250000,0 days 00:22:00,22.000000
max,3167.000000,59595.000000,0 days 00:32:00,32.000000


In [8]:
courier_actions.to_csv("courier_actions_new.csv")